# Imports <a id="imports"></a>
This cell will contain the imports necessary for the project

In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import numpy as np

"""
repair.py contains functions that helpes with repairing dataset and performing
data munging
"""
import repair 

In [2]:
df = pd.read_csv("googleplaystore.csv")

# The Dataset <a id="ds"></a> 
The dataset we chose is the ["Google Play Store Apps"](https://www.kaggle.com/lava18/google-play-store-apps) dataset with around 10,841 data rows.

## The Headings <a id="hd"></a> 
| App | Category | Rating | Reviews | Size | Installs | Type | Price | Content Rating | Genres | Last Updated | Current Ver | Android Ver |
|---|---|---|---|---|---|---|---|---|---|---|---|---|
| Facebook | SOCIAL | 4.1 | 78158306 | Varies with device | 1,000,000,000+ | Free | 0.0 | Teen | Social | August 3, 2018 | Varies with device | Varies with device |

### App
- Application name

### Category
- The category the app belongs to

### Rating
- Number of user reviews for the app (as when scraped)

### Reviews 
- Number of user reviews for the app (as when scraped)

### Installs 
- Number of user downloads/install for the app (as when scraped)

### Type
-  Paid or Free

### Price
- Price of the app (as when scraped)

### Content Rating
- Age group the app is targeted at - Children / Mature 21+ / Adult

### Genres
- An app can belong to multiple genres (apart from its main category). For example. a musical family game will belong to Music, Game, Family genres.

### Last Updated 
- Date when the app was last updated on the Play Store (as when scraped)

### Current Ver 
- Current version of the app available on Play Store (as when scraped)

### Android Ver
- Min required Android version (as when scraped)

# Data Munging

## Data types
When you look at the current version of the dataset, all of the columns are of type object:

In [3]:
df.dtypes

App                object
Category           object
Rating            float64
Reviews             int64
Size               object
Installs           object
Type               object
Price              object
Content Rating     object
Genres             object
Last Updated       object
Current Ver        object
Android Ver        object
dtype: object

We believed that the *"Reviews"* and *"Price"* columns should be of a numerical type. We would do this with the help of the function ```pandas.to_numeric()``` and the ```remove_dollar_sign()``` function in ```repair.py``` which removes the dollar sign from the prices.

In [4]:
# create new dataset of cleaned data of extracted dollar sign
repair.remove_dollar_sign("googleplaystore.csv", "googleplaystore2.csv")

repair.py: Printing done


In [5]:
# update the data frame
df = pd.read_csv("googleplaystore2.csv")

# convert Price to numerical
df["Price"] = pd.to_numeric(df["Price"])

In [6]:
# convert Reviews column to numerical
df["Reviews"] = pd.to_numeric(df["Reviews"])

In [7]:
df.dtypes

App                object
Category           object
Rating            float64
Reviews             int64
Size               object
Installs           object
Type               object
Price             float64
Content Rating     object
Genres             object
Last Updated       object
Current Ver        object
Android Ver        object
dtype: object

After the fixing the data, we have converted all columns that should be numeric into numerics

## Filling null values

Below displays the columns that contain null values

In [8]:
df.apply(lambda x: sum(x.isnull()),axis=0)

App                  0
Category             0
Rating            1474
Reviews              0
Size                 0
Installs             0
Type                 1
Price                0
Content Rating       0
Genres               1
Last Updated         0
Current Ver          8
Android Ver          2
dtype: int64

### Rating Column

In [9]:
# Get data rows where the value is null
null_values = df.loc[df["Rating"].isnull()]

In [10]:
null_values

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
23,Mcqueen Coloring pages,ART_AND_DESIGN,NaN,61,7.0M,"100,000+",Free,0.00,Everyone,Art & Design;Action & Adventure,"March 7, 2018",1.0.0,4.1 and up
113,Wrinkles and rejuvenation,BEAUTY,NaN,182,5.7M,"100,000+",Free,0.00,Everyone 10+,Beauty,"September 20, 2017",8.0,3.0 and up
123,Manicure - nail design,BEAUTY,NaN,119,3.7M,"50,000+",Free,0.00,Everyone,Beauty,"July 23, 2018",1.3,4.1 and up
126,Skin Care and Natural Beauty,BEAUTY,NaN,654,7.4M,"100,000+",Free,0.00,Teen,Beauty,"July 17, 2018",1.15,4.1 and up
129,"Secrets of beauty, youth and health",BEAUTY,NaN,77,2.9M,"10,000+",Free,0.00,Mature 17+,Beauty,"August 8, 2017",2.0,2.3 and up
130,Recipes and tips for losing weight,BEAUTY,NaN,35,3.1M,"10,000+",Free,0.00,Everyone 10+,Beauty,"December 11, 2017",2.0,3.0 and up
134,"Lady adviser (beauty, health)",BEAUTY,NaN,30,9.9M,"10,000+",Free,0.00,Mature 17+,Beauty,"January 24, 2018",3.0,3.0 and up
163,Anonymous caller detection,BOOKS_AND_REFERENCE,NaN,161,2.7M,"10,000+",Free,0.00,Everyone,Books & Reference,"July 13, 2018",1.0,2.3 and up
180,SH-02J Owner's Manual (Android 8.0),BOOKS_AND_REFERENCE,NaN,2,7.2M,"50,000+",Free,0.00,Everyone,Books & Reference,"June 15, 2018",3.0,6.0 and up
185,URBANO V 02 instruction manual,BOOKS_AND_REFERENCE,NaN,114,7.3M,"100,000+",Free,0.00,Everyone,Books & Reference,"August 7, 2015",1.1,5.1 and up


In [11]:
df.apply(lambda x: sum(x.isnull()),axis=0) 

App                  0
Category             0
Rating            1474
Reviews              0
Size                 0
Installs             0
Type                 1
Price                0
Content Rating       0
Genres               1
Last Updated         0
Current Ver          8
Android Ver          2
dtype: int64

According from the data above, there are 1474 rows with null values for **Rating**. This was quite problematic as this field is key for our predictive modelling later on. It does not make it any better that these null values make up about %13.5 of the total dataset. There are options to reduce the impact and the two biggest are to get the mean of the Rating column or to just delete them. There are obvious problems to these:
1. Deleting all 1474 ratings may have an effect of the performance of the maching learning algorithms later which is made way worse by the fact that the Ratings column will be a key attribute that we will be using for the predictives model later
2. Getting the mean of the entire  Rating column may not accurately represent the true rating for the null data rows

There are a couple of workarounds that we have thought of:
- Get the mean of non null values for each category and use them to to replace the null values of the categories
- If there are very little number of null value rows for a category, they will be deleted

In [12]:
def fill_null_ratings_for_category(df):
    # Get all unique values for the category column
    categories = df["Category"].unique().tolist()
    
    for category in categories:
        cat = df.loc[df["Category"] == category]
        
        # get rows of current category where ratings is not null 
        cat_not_null = cat.loc[cat["Rating"].notna()]
        avg = cat_not_null["Rating"].mean()
        
        df.loc[df["Category"] == category, "Rating"] = df.loc[df["Category"] == category, "Rating"].fillna(avg)
    
    
        
    

fill_null_ratings_for_category(df)

In [13]:
df.apply(lambda x: sum(x.isnull()),axis=0) 

App               0
Category          0
Rating            0
Reviews           0
Size              0
Installs          0
Type              1
Price             0
Content Rating    0
Genres            1
Last Updated      0
Current Ver       8
Android Ver       2
dtype: int64

From the above, we can conclude that the function we created successfully filled in the missing values in the rating column. The next thing we must do is to remove the null values from the rest of the columns

### Type

In [14]:
df.loc[df["Type"].isnull()]

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
9148,Command & Conquer: Rivals,FAMILY,4.192272,0,Varies with device,0,NaN,0.0,Everyone 10+,Strategy,"June 28, 2018",Varies with device,Varies with device


In this scenario, it is very easy to fill in the null value. As the **Price** for this app is 0.0, the value for the **Type** should be **"Free"**

In [15]:
df["Type"].fillna("Free", inplace=True)

In [16]:
df.apply(lambda x: sum(x.isnull()),axis=0) 

App               0
Category          0
Rating            0
Reviews           0
Size              0
Installs          0
Type              0
Price             0
Content Rating    0
Genres            1
Last Updated      0
Current Ver       8
Android Ver       2
dtype: int64

### Genres

In [17]:
df.loc[df["Genres"].isnull()]

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
10472,Life Made WI-Fi Touchscreen Photo Frame,PERSONALIZATION,1.9,19,3M,"1,000+",Free,0.0,Everyone,NaN,"February 11, 2018",1.0.19,4.0 and up


This is yet another simple problem to find the solution to. As the main category for the app is "PERSONALIZATION", we can conclude that the genre for the app is "Personalization".

In [18]:
df["Genres"].fillna("Personalization", inplace=True)

### Current Ver

In [19]:
df.loc[df["Current Ver"].isnull()]

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
15,Learn To Draw Kawaii Characters,ART_AND_DESIGN,3.200000,55,2.7M,"5,000+",Free,0.00,Everyone,Art & Design,"June 6, 2018",NaN,4.2 and up
1553,Market Update Helper,LIBRARIES_AND_DEMO,4.100000,20145,11k,"1,000,000+",Free,0.00,Everyone,Libraries & Demo,"February 12, 2013",NaN,1.5 and up
6322,Virtual DJ Sound Mixer,TOOLS,4.200000,4010,8.7M,"500,000+",Free,0.00,Everyone,Tools,"May 10, 2017",NaN,4.0 and up
6803,BT Master,FAMILY,4.192272,0,222k,100+,Free,0.00,Everyone,Education,"November 6, 2016",NaN,1.6 and up
7333,Dots puzzle,FAMILY,4.000000,179,14M,"50,000+",Paid,0.99,Everyone,Puzzle,"April 18, 2018",NaN,4.0 and up
7407,Calculate My IQ,FAMILY,4.192272,44,7.2M,"10,000+",Free,0.00,Everyone,Entertainment,"April 3, 2017",NaN,2.3 and up
7730,UFO-CQ,TOOLS,4.047411,1,237k,10+,Paid,0.99,Everyone,Tools,"July 4, 2016",NaN,2.0 and up
10342,La Fe de Jesus,BOOKS_AND_REFERENCE,4.346067,8,658k,"1,000+",Free,0.00,Everyone,Books & Reference,"January 31, 2017",NaN,3.0 and up


In [20]:
df["Current Ver"].value_counts()

Varies with device    1459
1.0                    809
1.1                    264
1.2                    178
2.0                    151
1.3                    145
1.0.0                  136
1.0.1                  119
1.4                     88
1.5                     81
1.0.2                   80
1.6                     65
1.0.3                   62
2.1                     61
3.0                     59
1.0.4                   58
1.7                     53
2.0.0                   51
1.0.5                   50
1.1.0                   49
1.1.1                   48
1.0.6                   47
1.2.1                   46
1.2.0                   43
1.8                     42
4.0                     40
1.9                     37
2.3.2                   35
1.0.7                   34
2.4                     34
                      ... 
1.0.0.17                 1
1.2.10                   1
3.21.324                 1
4.9.5                    1
7.9.0 (5192)             1
2.8.7                    1
5

We have concluded that it would be most appropriate to fill the null values with **"Varies with device"** as there is a likely probability that it would have that version.

In [21]:
df["Current Ver"].fillna("Varies with device", inplace=True)

### Android Ver

In [22]:
df.loc[df["Android Ver"].isnull()]

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
4453,[substratum] Vacuum: P,PERSONALIZATION,4.4,230,11M,"1,000+",Paid,1.49,Everyone,Personalization,"July 20, 2018",4.4,NaN
4490,Pi Dark [substratum],PERSONALIZATION,4.5,189,2.1M,"10,000+",Free,0.00,Everyone,Personalization,"March 27, 2018",1.1,NaN


In [23]:
df["Android Ver"].value_counts()

4.1 and up            2451
4.0.3 and up          1501
4.0 and up            1376
Varies with device    1362
4.4 and up             980
2.3 and up             652
5.0 and up             601
4.2 and up             394
2.3.3 and up           281
2.2 and up             244
4.3 and up             243
3.0 and up             241
2.1 and up             134
1.6 and up             116
6.0 and up              60
7.0 and up              42
3.2 and up              36
2.0 and up              32
5.1 and up              24
1.5 and up              20
4.4W and up             12
3.1 and up              10
2.0.1 and up             7
8.0 and up               6
7.1 and up               3
4.0.3 - 7.1.1            2
1.0 and up               2
5.0 - 8.0                2
5.0 - 7.1.1              1
7.0 - 7.1.1              1
5.0 - 6.0                1
2.2 - 7.1.1              1
4.1 - 7.1.1              1
Name: Android Ver, dtype: int64

We have concluded that **"Varies with device"** is the most appropriate value to fill the values as doing so would be safer than labelling it with a definite android version.

In [24]:
df["Android Ver"].fillna("Varies with device", inplace=True)

In [25]:
df.apply(lambda x: sum(x.isnull()),axis=0) 

App               0
Category          0
Rating            0
Reviews           0
Size              0
Installs          0
Type              0
Price             0
Content Rating    0
Genres            0
Last Updated      0
Current Ver       0
Android Ver       0
dtype: int64

We have now finished with filling the null values! **BOOM :D**

In [26]:
from sklearn.model_selection import KFold
def classification_model(model, data, predictors, outcome):
    model.fit(data[predictors],data[outcome])
    predictions = model.predict(data[predictors])
    accuracy = metrics.accuracy_score(predictions,data[outcome])
    print ("Accuracy : %s" % "{0:.3%}".format(accuracy))
    kf = KFold(data.shape[0], n_splits=10)
    error = []
    for train, test in kf:
        train_predictors = (data[predictors].iloc[train,:])
        train_target = data[outcome].iloc[train]
        model.fit(train_predictors, train_target)
        error.append(model.score(data[predictors].iloc[test,:], data[outcome].iloc[test]))
    print ("Cross-Validation Score : %s" % "{0:.3%}".format(np.mean(error)))
    model.fit(data[predictors],data[outcome])

In [27]:
for train_index, test_index in kf.split(X):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    ('TRAIN:', array([2, 3]), 'TEST:', array([0, 1]))
    ('TRAIN:', array([0, 1]), 'TEST:', array([2, 3]))

NameError: name 'kf' is not defined